In [1]:
import os
os.chdir('/home/composersyf/Documents/Political Data Science Project')

In [2]:
import numpy as np
import pandas as pd
import re

In [5]:
import glob
trump_filenames=glob.glob("TwitterData/* -at,realDonaldTrump*")
trump_filenames_table=[]
for fname in trump_filenames:
    trump_filenames_table.append(fname)
    trump_filenames_table.extend(fname.split("]")[1].split(".")[:3])
trump_filenames_table=np.array(trump_filenames_table)
trump_filenames_table=trump_filenames_table.reshape(len(trump_filenames_table)//4,4)
trump_filenames_table=pd.DataFrame(trump_filenames_table)
trump_filenames_table.columns=["filename","year","month","day"]
trump_filenames_table.day=trump_filenames_table["day"].astype(int)
trump_filenames_table.month=trump_filenames_table["month"].astype(int)
trump_filenames_table.year=trump_filenames_table["year"].astype(int)
trump_filenames_table=trump_filenames_table.sort_values(["year","month","day"])
trump_filenames_table

,filename,year,month,day
72,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,19
92,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,19
161,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,19
228,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,19
1,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,20
19,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,20
24,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,20
27,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,20
29,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,20
30,"TwitterData/[ BATCH -at,realDonaldTrump-STATUS...",2016,5,20


In [23]:
from nltk.tokenize import TweetTokenizer
import random

days=np.unique(trump_filenames_table.day)

tweet_sample_trump=[]

for d in days: #loop through the entire week, on a daily basis
    fnames=np.array(trump_filenames_table[trump_filenames_table.day==d].iloc[:,0])
    all_tweets=[]
    all_usernames=[]
    for f in fnames:
        with open(f) as file:
            file_texts=file.readlines()
            file_texts=file_texts[1:]
        tweets=[None]*len(file_texts)
        usernames=[None]*len(file_texts)
        for i,text in enumerate(file_texts):
            splitted_text=text.split("\",\"")
            tweets[i]=splitted_text[11]
            usernames[i]=splitted_text[6]
        all_tweets.extend(tweets)
        all_usernames.extend(usernames)
    tweets_data=np.array(all_usernames+all_tweets)
    tweets_data=tweets_data.reshape(2,len(all_tweets))
    tweets_data=tweets_data.T
    #remove duplicates
    tweets_data_df=pd.DataFrame(tweets_data)
    tweets_data_df=tweets_data_df.drop_duplicates()
    total_tweet_counts[str(d)]=tweets_data_df.shape[0]
    
    tknzr = TweetTokenizer()
    at_regex=re.compile("^@")
    good_tweets=[]
    #count the number of tweets that contain both @realDonaldTrump and @ other candidates (particularly @HillaryClinton)
    for i in range(tweets_data_df.shape[0]):
        tokens=tknzr.tokenize(tweets_data_df.iloc[i,1])
        if tokens[0]=="RT":
            continue
        add_tweet=True
        for token in tokens:
            if at_regex.findall(token)==["@"] and token.lower() in ['@berniesanders', '@hillaryclinton','@johnkasich','@senrubiopress','@sensanders','@tedcruz']:               
                add_tweet=False
                break
        if add_tweet==True:
            good_tweets.append(tweets_data_df.iloc[i,1])
    tweet_sample_trump.extend(random.sample(good_tweets,250)[:])

print(len(tweet_sample_trump))

3000


In [32]:
tweet_sample_trump=np.array(tweet_sample_trump)
tweet_sample_trump=pd.DataFrame(tweet_sample_trump)
tweet_sample_trump.to_csv("tweet_sample_trump.csv",header=False,index=False)

In [33]:
clinton_filenames=glob.glob("TwitterData/* -at,HillaryClinton*")
clinton_filenames_table=[]
for fname in clinton_filenames:
    clinton_filenames_table.append(fname)
    clinton_filenames_table.extend(fname.split("]")[1].split(".")[:3])
clinton_filenames_table=np.array(clinton_filenames_table)
clinton_filenames_table=clinton_filenames_table.reshape(len(clinton_filenames_table)//4,4)
clinton_filenames_table=pd.DataFrame(clinton_filenames_table)
clinton_filenames_table.columns=["filename","year","month","day"]
clinton_filenames_table.day=clinton_filenames_table["day"].astype(int)
clinton_filenames_table.month=clinton_filenames_table["month"].astype(int)
clinton_filenames_table.year=clinton_filenames_table["year"].astype(int)
clinton_filenames_table=clinton_filenames_table.sort_values(["year","month","day"])
clinton_filenames_table

,filename,year,month,day
13,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,19
60,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,19
69,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,19
92,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,19
2,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,20
3,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,20
8,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,20
10,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,20
15,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,20
16,"TwitterData/[ BATCH -at,HillaryClinton-STATUS]...",2016,5,20


In [36]:
days=np.unique(clinton_filenames_table.day)

tweet_sample_clinton=[]

for d in days: #loop through the entire week, on a daily basis
    fnames=np.array(clinton_filenames_table[clinton_filenames_table.day==d].iloc[:,0])
    all_tweets=[]
    all_usernames=[]
    for f in fnames:
        with open(f) as file:
            file_texts=file.readlines()
            file_texts=file_texts[1:]
        tweets=[None]*len(file_texts)
        usernames=[None]*len(file_texts)
        for i,text in enumerate(file_texts):
            splitted_text=text.split("\",\"")
            tweets[i]=splitted_text[11]
            usernames[i]=splitted_text[6]
        all_tweets.extend(tweets)
        all_usernames.extend(usernames)
    tweets_data=np.array(all_usernames+all_tweets)
    tweets_data=tweets_data.reshape(2,len(all_tweets))
    tweets_data=tweets_data.T
    #remove duplicates
    tweets_data_df=pd.DataFrame(tweets_data)
    tweets_data_df=tweets_data_df.drop_duplicates()
    total_tweet_counts[str(d)]=tweets_data_df.shape[0]
    
    #random sampling for each day, keeping the sample size of each day equal to each other
    tknzr = TweetTokenizer()
    at_regex=re.compile("^@")
    good_tweets=[]
    for i in range(tweets_data_df.shape[0]):
        tokens=tknzr.tokenize(tweets_data_df.iloc[i,1])
        if tokens[0]=="RT":
            continue
        add_tweet=True
        for token in tokens:
            if at_regex.findall(token)==["@"] and token.lower() in ['@berniesanders', '@realdonaldtrump','@johnkasich','@senrubiopress','@sensanders','@tedcruz']:               
                add_tweet=False
                break
        if add_tweet==True:
            good_tweets.append(tweets_data_df.iloc[i,1])
    if d==1:
        tweet_sample_clinton.extend(random.sample(good_tweets,280)[:])
    else:
        tweet_sample_clinton.extend(random.sample(good_tweets,272)[:])

print(len(tweet_sample_clinton))

3000


In [37]:
tweet_sample_clinton=np.array(tweet_sample_clinton)
tweet_sample_clinton=pd.DataFrame(tweet_sample_clinton)
tweet_sample_clinton.to_csv("tweet_sample_clinton.csv",header=False,index=False)

In [39]:
sanders_filenames_1=glob.glob("TwitterData/* -at,BernieSanders*")
sanders_filenames_2=glob.glob("TwitterData/* -at,SenSanders*")
sanders_filenames=sanders_filenames_1+sanders_filenames_2
sanders_filenames_table=[]
for fname in sanders_filenames:
    sanders_filenames_table.append(fname)
    sanders_filenames_table.extend(fname.split("]")[1].split(".")[:3])
sanders_filenames_table=np.array(sanders_filenames_table)
sanders_filenames_table=sanders_filenames_table.reshape(len(sanders_filenames_table)//4,4)
sanders_filenames_table=pd.DataFrame(sanders_filenames_table)
sanders_filenames_table.columns=["filename","year","month","day"]
sanders_filenames_table.day=sanders_filenames_table["day"].astype(int)
sanders_filenames_table.month=sanders_filenames_table["month"].astype(int)
sanders_filenames_table.year=sanders_filenames_table["year"].astype(int)
sanders_filenames_table=sanders_filenames_table.sort_values(["year","month","day"])
sanders_filenames_table

,filename,year,month,day
47,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
53,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
67,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
69,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
91,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
96,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
98,"TwitterData/[ BATCH -at,BernieSanders-STATUS]2...",2016,5,19
113,"TwitterData/[ BATCH -at,SenSanders-STATUS]2016...",2016,5,19
140,"TwitterData/[ BATCH -at,SenSanders-STATUS]2016...",2016,5,19
146,"TwitterData/[ BATCH -at,SenSanders-STATUS]2016...",2016,5,19


In [41]:
days=np.unique(sanders_filenames_table.day)

tweet_sample_sanders=[]

for d in days: #loop through the entire week, on a daily basis
    fnames=np.array(sanders_filenames_table[sanders_filenames_table.day==d].iloc[:,0])
    all_tweets=[]
    all_usernames=[]
    for f in fnames:
        with open(f) as file:
            file_texts=file.readlines()
            file_texts=file_texts[1:]
        tweets=[None]*len(file_texts)
        usernames=[None]*len(file_texts)
        for i,text in enumerate(file_texts):
            splitted_text=text.split("\",\"")
            tweets[i]=splitted_text[11]
            usernames[i]=splitted_text[6]
        all_tweets.extend(tweets)
        all_usernames.extend(usernames)
    tweets_data=np.array(all_usernames+all_tweets)
    tweets_data=tweets_data.reshape(2,len(all_tweets))
    tweets_data=tweets_data.T
    #remove duplicates
    tweets_data_df=pd.DataFrame(tweets_data)
    tweets_data_df=tweets_data_df.drop_duplicates()
    total_tweet_counts[str(d)]=tweets_data_df.shape[0]
    
    tknzr = TweetTokenizer()
    at_regex=re.compile("^@")
    good_tweets=[]
    for i in range(tweets_data_df.shape[0]):
        tokens=tknzr.tokenize(tweets_data_df.iloc[i,1])
        if tokens[0]=="RT":
            continue
        add_tweet=True
        for token in tokens:
            if at_regex.findall(token)==["@"] and token.lower() in ['@realdonaldtrump', '@hillaryclinton','@johnkasich','@senrubiopress','@tedcruz']:               
                add_tweet=False
                break
        if add_tweet==True:
            good_tweets.append(tweets_data_df.iloc[i,1])
    tweet_sample_sanders.extend(random.sample(good_tweets,250)[:])

print(len(tweet_sample_sanders))

3000


In [42]:
tweet_sample_sanders=np.array(tweet_sample_sanders)
tweet_sample_sanders=pd.DataFrame(tweet_sample_sanders)
tweet_sample_sanders.to_csv("tweet_sample_sanders.csv",header=False,index=False)